In [137]:
import argparse
from IPython import embed

import time
import numpy as np
import pyro
import pyro.distributions as dist
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd.variable as Variable

import torch_geometric.transforms as T
import torch_geometric.utils as torch_util
import torch_scatter

from torch_geometric.data import DataLoader
from torch_geometric.datasets import MNISTSuperpixels, Planetoid
from torch_geometric.nn import ChebConv, GCNConv, SAGEConv

from pyro.infer import SVI, JitTrace_ELBO, Trace_ELBO
from pyro.optim import Adam

import networkx as nx
import visdom
import scipy as sp

from collections import defaultdict

torch.set_default_tensor_type('torch.FloatTensor')

In [466]:
# define the PyTorch module that parameterizes the
# diagonal gaussian distribution q(z|x)
class Encoder(nn.Module):
    def __init__(self, n_feat, hidden_dim, latent_dim, dropout):
        super(Encoder, self).__init__()

        # Set up thhe Graph convolutional layers 
        self.gc1 = GCNConv(n_feat, hidden_dim)
        self.gc2_mu = GCNConv(hidden_dim, latent_dim)
        self.gc2_sig = GCNConv(hidden_dim, latent_dim)
        
        # self.gc1 = SAGEConv(n_feat, hidden_dim)
        # self.gc2_mu = SAGEConv(hidden_dim, z_dim)
        # self.gc2_sig = SAGEConv(hidden_dim, z_dim)    
        
        # Setup for non-linearities
        self.softplus = nn.Softplus()
        self.relu = torch.nn.ReLU()
        
        # Dropout
        self.dropout = dropout

    def forward(self, x, adj):
        # define the forward computation on the adjacency matrix for each graph and its features, x 

        print('Features: ', x, x.shape)
        print("Adj: ", adj, adj.shape)
        
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        
        # Sharing parameters between mu and sigma 
        z_loc = self.gc2_mu(x, adj)
        z_scale = torch.exp(self.gc2_sig(x, adj))

#         hidden = self.softplus(self.gc1(x, adj))
#         z_loc = self.gc2_mu(hidden, adj)
#         z_scale = torch.exp(self.gc2_sig(hidden, adj))

        return z_loc, z_scale


# TODO: Modify from Inner Product decoder 
# define the PyTorch module that parameterizes the observation likelihood p(x|z)

# class Decoder(nn.Module):
#     def __init__(self, z_dim, hidden_dim):
#         super(Decoder, self).__init__()
#         # setup the two linear transformations used for the decoder 
#         self.fc1 = nn.Linear(z_dim, hidden_dim)
#         self.fc21 = nn.Linear(hidden_dim, 75)
#         # setup the non-linearities
#         self.softplus = nn.Softplus()

#     def forward(self, z):
#         # define the forward computation on the latent z
#         # first compute the hidden units

#         hidden = self.softplus(self.fc1(z))

#         # return the parameter for the output Bernoulli
#         loc_img = torch.sigmoid(self.fc21(hidden))
#         return loc_img

class Decoder(nn.Module):
    def __init__(self, dropout):
        super(Decoder, self).__init__()
        self.dropout = dropout
        self.fudge = 1e-7
    
    def forward(self, z): 
        z = F.dropout(z, self.dropout, training=self.training)
        adj = (nn.Sigmoid(torch.mm(z, z.t())) + self.fudge) * (1 - 2 * self.fudge)

In [467]:
class VGAE(nn.Module):
    """Graph Auto Encoder (see: https://arxiv.org/abs/1611.07308)"""

    def __init__(self, data, n_hidden, n_latent, dropout, subsampling=False):
        super(VGAE, self).__init__()

        # Data
        self.x = data['features']
        self.adj_norm = data['adj_norm']
        self.adj_labels = data['adj_labels']
        self.obs = self.adj_labels.view(1, -1)

        # Dimensions
        N, D = data['features'].shape
        self.n_samples = N
        self.n_edges = self.adj_labels.sum()
        self.n_subsample = 2 * self.n_edges
        self.input_dim = D
        self.n_hidden = n_hidden
        self.n_latent = n_latent

        # Parameters
        self.pos_weight = float(N * N - self.n_edges) / self.n_edges
        self.norm = float(N * N) / ((N * N - self.n_edges) * 2)
        self.subsampling = subsampling

        # Layers
        self.dropout = dropout
        self.encoder = Encoder(self.input_dim, self.n_hidden, self.n_latent, self.dropout)
        self.decoder = Decoder(self.dropout)


    def model(self):
        # register PyTorch module `decoder` with Pyro
        pyro.module("decoder", self.decoder)

        # Setup hyperparameters for prior p(z)
        z_mu    = torch.zeros([self.n_samples, self.n_latent])
        z_sigma = torch.ones([self.n_samples, self.n_latent])

        # sample from prior
        z = pyro.sample("latent", dist.Normal(z_mu, z_sigma).to_event(2))

        # decode the latent code z
        z_adj = self.decoder(z).view(1, -1)

        # Score against data
        pyro.sample('obs', WeightedBernoulli(z_adj, weight=self.pos_weight).to_event(2), obs=self.obs)


    def guide(self):
        # register PyTorch model 'encoder' w/ pyro
        pyro.module("encoder", self.encoder)

        # Use the encoder to get the parameters use to define q(z|x)
        print(self.x.shape)
        print(self.adj_norm.shape)
        print(self.x)
        print(self.adj_norm)
                
        z_mu, z_sigma = self.encoder(self.x, self.adj_norm)

        # Sample the latent code z
        pyro.sample("latent", dist.Normal(z_mu, z_sigma).to_event(2))


    def get_embeddings(self):
        z_mu, _ = self.encoder.eval()(self.x, self.adj_norm)
        # Put encoder back into training mode
        self.encoder.train()
        return z_mu

In [468]:
##TODO: Get Cora data
def get_data():
    dataset = args.name
    
    if dataset == 'MNIST':
        path = '../data/geometric/MNIST'
        trainset = MNISTSuperpixels(path, train=True)
        testset = MNISTSuperpixels(path, train=False)

        lenTrain = len(trainset)
        lenTest = len(testset)

        trainLoader = DataLoader(trainset[:lenTrain//125], batch_size=1, shuffle=False)
        testloader = DataLoader(testset[:lenTest//125], batch_size=1, shuffle=False)
        return trainLoader, testloader

    elif dataset == 'CORA':
        dataset = 'Cora'
        path = path = '../data/geometric/CORA'
        dataset = Planetoid(path, dataset, T.NormalizeFeatures())

        return dataset, _


In [469]:
# Load in Data 
print('Using {} dataset'.format(args.name))

np.random.seed(0)

train_loader, test_loader = get_data()
data = train_loader[0]

# Get features and store their shape
features = np.array(data['x'])
N, D = features.shape

edgeList = np.array(data['edge_index'].transpose(1,0 ))
adj = nx.adjacency_matrix(nx.from_edgelist(edgeList))    

adj_original = adj
adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges = mask_edges(adj)

adj_train_norm   = preprocess_graph(adj_train)
adj_train_norm   = Variable(make_sparse(adj_train_norm))
adj_train_labels = Variable(torch.FloatTensor(adj_train + sp.sparse.eye(adj_train.shape[0]).todense()))

features = Variable(make_sparse(features))
n_edges = adj_train_labels.sum()

Using CORA dataset
preprocess finished


In [470]:

# feat = data['features']
# adj = data['adj_norm']

# feat._values().resize_((feat._values().shape[0], 1))


# # print(feat._values())
# # print(adj._indices().shape)

# featOut = feat._values()
# adjOut = adj._indices()

# print(featOut, featOut.shape)
# print(adjOut, adjOut.shape)

feat = features._values().resize_((features._values().shape[0], 1))
print(feat.shape)
print(adj_train_norm._indices().shape)

# data = {
#     'adj_norm'  : adj_train_norm,
#     'adj_labels': adj_train_labels,
#     'features'  : features,
# }

data = {
    'adj_norm'  : adj_train_norm._indices(),
    'adj_labels': adj_train_labels,
    'features'  : features._values().resize_((features._values().shape[0], 1)),
}

# print(data['adj_norm'])
# print(data['features'])

vgae = VGAE(data, 
            n_hidden=32,
            n_latent=16,
            dropout=args.dropout
           )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model, data = VGAE().to(device), data.to(device)
    
optimizer = Adam({"lr": args.lr, "betas": (0.95, 0.999)})


svi = SVI(vgae.model, vgae.guide, optimizer, loss=Trace_ELBO())
results = defaultdict(list)

torch.Size([49216, 1])
torch.Size([2, 11684])


In [471]:
for epoch in range(args.num_epochs):
    # initialize loss accumulator
    epoch_loss = 0.
    # do ELBO gradient and accumulate loss
    epoch_loss += svi.step()

    # report training diagnostics
    normalized_loss = epoch_loss / (2 * N * N)

    results['train_elbo'].append(normalized_loss)

    # Training loss
    emb = gae.get_embeddings()

    accuracy, roc_curr, ap_curr = eval_gae(val_edges, val_edges_false, emb, adj_orig)

    results['accuracy_train'].append(accuracy)
    results['roc_train'].append(roc_curr)
    results['ap_train'].append(ap_curr)

    print("Epoch:", '%04d' % (epoch + 1),
          "train_loss=", "{:.5f}".format(normalized_loss),
          "train_acc=", "{:.5f}".format(accuracy), "val_roc=", "{:.5f}".format(roc_curr), "val_ap=", "{:.5f}".format(ap_curr))

    # Test loss
    if epoch % args.test_freq == 0:
        emb = gae.get_embeddings()
        accuracy, roc_score, ap_score = eval_gae(test_edges, test_edges_false, emb, adj_orig)
        results['accuracy_test'].append(accuracy)
        results['roc_test'].append(roc_curr)
        results['ap_test'].append(ap_curr)

torch.Size([49216, 1])
torch.Size([2, 11684])
tensor([[0.1111],
        [0.1111],
        [0.1111],
        ...,
        [0.0769],
        [0.0769],
        [0.0769]])
tensor([[   0,    0,    0,  ..., 2706, 2707, 2707],
        [   3,    2,    1,  ..., 2706, 2707,  840]])
Features:  tensor([[0.1111],
        [0.1111],
        [0.1111],
        ...,
        [0.0769],
        [0.0769],
        [0.0769]]) torch.Size([49216, 1])
Adj:  tensor([[   0,    0,    0,  ..., 2706, 2707, 2707],
        [   3,    2,    1,  ..., 2706, 2707,  840]]) torch.Size([2, 11684])


KeyboardInterrupt: 

In [472]:
assert pyro.__version__.startswith('0.3.1')
# parse command line arguments
parser = argparse.ArgumentParser(description="parse args")
parser.add_argument('--num-epochs', default=2, type=int, help='number of training epochs')
parser.add_argument('--tf', '--test-frequency', default=5, type=int, help='how often we evaluate the test set')
parser.add_argument('--lr', '--learning-rate', default=1.0e-3, type=float, help='learning rate')
    
parser.add_argument('--cuda', action='store_true', default=False, help='whether to use cuda')
parser.add_argument('--jit', action='store_true', default=False, help='whether to use PyTorch jit')
parser.add_argument('-visdom', '--visdom_flag', action="store_true", help='Whether plotting in visdom is desired')
parser.add_argument('--time', default=int(time.time()), help="Current system time")

parser.add_argument('--name', default='CORA', help="Name of the dataset")
parser.add_argument('--save', default=False, help="Whether to save the trained model")
parser.add_argument('--dropout', default=0, help="Dropout probability")
args = parser.parse_args(args=[])

# main(args)

In [473]:



# # adj2 = nx.adjacency_matrix(nx.from_edgelist(np.array(adj._indices().transpose(1, 0))))
# # print(torch.sparse.FloatTensor(adj2.tocoo()))
# # edgeList = np.array(data['edge_index'].transpose(1,0 ))
# gc = GCNConv(D, 1)

# out = gc(featOut, adjOut)

# # torch.Size([75, 1])
# # Adj     :  tensor([[ 0,  0,  0,  ..., 74, 74, 74],
# #         [ 3,  8, 10,  ..., 55, 63, 69]]) torch.Size([2, 1399])

In [474]:
# def preprocess_graph(adj):
#     adj = sp.sparse.coo_matrix(adj)
#     adj_ = adj + sp.eye(adj.shape[0])
#     rowsum = np.array(adj_.sum(1))
#     degree_mat_inv_sqrt = sp.sparse.diags(np.power(rowsum, -0.5).flatten())
#     adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(
#         degree_mat_inv_sqrt).tocoo()
#     return adj_normalized

def preprocess_graph(adj):
    adj = sp.sparse.coo_matrix(adj)
    adj_ = adj + sp.sparse.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.sparse.diags(np.power(rowsum, -.5).flatten())
    coodot = adj_.dot(degree_mat_inv_sqrt).tocoo()
    adj_normalized = coodot.transpose().dot(degree_mat_inv_sqrt).tocoo()
    print('preprocess finished')
    return adj_normalized
    

def make_sparse(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
#     sparse_mx = sparse_mx.tocoo().astype(np.float32)
    sparse_mx = sp.sparse.coo_matrix(sparse_mx, dtype=np.float32)
    indices = torch.Tensor(np.array(np.vstack((sparse_mx.row, sparse_mx.col)), dtype=np.float32)).long()

    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.sparse.FloatTensor(torch.from_numpy(sparse_mx.data))
#     values = torch.from_numpy(sparse_mx.data)

    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
#     return torch.FloatTensor(indices, values, shape)


def sparse_to_tuple(sparse_mx):
    if not sp.sparse.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape
def ismember(a, b, tol=5):
    rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
    return np.any(rows_close)

def mask_edges(adj):
    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    # TODO: Clean up.

    # Remove diagonal elements
    adj = adj - \
        sp.sparse.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = sp.sparse.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.))
    num_val = int(np.floor(edges.shape[0] / 20.))

    all_edge_idx = list(range(edges.shape[0]))
    
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack(
        [test_edge_idx, val_edge_idx]), axis=0)


    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    assert ~ismember(test_edges_false, edges_all)
    assert ~ismember(val_edges_false, edges_all)
    assert ~ismember(val_edges, train_edges)
    assert ~ismember(test_edges, train_edges)
    assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.sparse.csr_matrix(
        (data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

def eval_gae(edges_pos, edges_neg, emb, adj_orig):

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    emb = emb.data.numpy()
    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []

    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []

    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])

    accuracy = accuracy_score((preds_all > 0.5).astype(float), labels_all)
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return accuracy, roc_score, ap_score